# web TA

## Penjelasan Setiap Cell pada Notebook
1. **Import Library**: Mengimpor semua library yang dibutuhkan untuk scraping dan pengolahan data.
2. **BASE_URL**: Mendefinisikan URL dasar untuk scraping data skripsi.
3. **get_max_page**: (Opsional) Fungsi untuk mencari jumlah halaman maksimal per prodi.
4. **print_progress**: Fungsi untuk menampilkan progress bar di terminal.
5. **crawl_detail & pta_all**: Fungsi utama untuk mengambil detail skripsi dan melakukan scraping semua prodi, hasil disimpan ke CSV.
6. **Menjalankan pta_all()**: Memulai proses scraping dan penyimpanan data.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time
from concurrent.futures import ThreadPoolExecutor, as_completed

### Penjelasan Fungsi Library yang Digunakan
- **requests**: Untuk melakukan HTTP request (mengambil halaman web dari internet).
- **BeautifulSoup**: Untuk parsing dan mengekstrak data dari HTML (memudahkan mengambil elemen tertentu dari halaman web).
- **pandas**: Untuk mengolah, menyimpan, dan menampilkan data dalam bentuk tabel (DataFrame).
- **re**: Untuk pencocokan pola string menggunakan regular expression (regex).
- **sys**: Untuk mengakses fungsi sistem, seperti menulis output ke terminal secara langsung.
- **time**: Untuk mengatur jeda waktu (delay) dan mengukur durasi eksekusi.
- **ThreadPoolExecutor, as_completed**: Untuk menjalankan beberapa proses scraping secara paralel (multithreading), sehingga lebih cepat.

In [21]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

In [22]:
def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        max_page = int(href.split("/")[-1])
        return max_page
    return 1

#### Penjelasan Fungsi get_max_page
Fungsi ini digunakan untuk mencari jumlah halaman maksimal dari daftar skripsi pada setiap prodi di website PTA Trunojoyo.
- Melakukan request ke halaman pertama prodi tertentu.
- Melakukan parsing HTML untuk mencari tombol navigasi ke halaman terakhir (»).
- Jika ditemukan, mengambil nomor halaman terakhir dari link tersebut.
- Jika tidak ditemukan, diasumsikan hanya ada 1 halaman.

In [23]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

#### Penjelasan Fungsi print_progress
Fungsi ini digunakan untuk menampilkan progress bar di terminal saat proses scraping berjalan.
- Menghitung persentase progress berdasarkan halaman saat ini dan total halaman.
- Membuat visualisasi progress bar dengan karakter blok dan strip.
- Menampilkan progress bar, nomor prodi, nama prodi, dan persentase ke terminal secara real-time.
- Jika sudah sampai halaman terakhir, menambah baris baru agar output rapi.

In [ ]:
# fungsi ambil detail skripsi
def crawl_detail(link_keluar, prodi):
    try:
        # Ambil ID skripsi dari link
        id_match = re.search(r"/detail/(\d+)", link_keluar)
        pta_id = id_match.group(1) if id_match else None

        # Request halaman detail skripsi
        response = requests.get(link_keluar, timeout=10)
        soup1 = BeautifulSoup(response.content, "html.parser")
        isi = soup1.select_one('div#content_journal')

        # Ambil judul skripsi
        judul = isi.select_one('a.title').text.strip()
        # Ambil nama penulis
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
        # Ambil nama pembimbing I dan II
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

        # Ambil abstrak (Indonesia & Inggris)
        paragraf = isi.select('p[align="justify"]')
        abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
        abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

        # Return hasil dalam bentuk dict
        return {
            "id": pta_id,
            "penulis": penulis,
            "judul": judul,
            "abstrak_id": abstrak_id,
            "abstrak_en": abstrak_en,
            "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua,
            "prodi": prodi,
        }
    except Exception as e:
        # Jika gagal, return None
        return None

def pta_all():
    # Mulai timer
    start_time = time.time()
    results = []

    total_prodi = 41

    # Loop untuk setiap prodi
    for i in range(1, total_prodi + 1):
        prodi_results = []
        # Ambil hanya halaman pertama tiap prodi
        url = f"{BASE_URL}/{i}/1"  # hanya halaman pertama
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        jurnals = soup.select('li[data-cat="#luxury"]')

        # Cek apakah ada data
        isii = soup.select_one('div#begin')
        if not isii:
            continue
        prodi_full = isii.select_one('h2').text.strip()
        prodi = prodi_full.replace("Journal Jurusan ", "")

        # Ambil semua link skripsi di halaman
        links = [jurnal.select_one('a.gray.button')['href'] for jurnal in jurnals]

        # Ambil detail skripsi secara paralel, maksimal 5 data
        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_link = {executor.submit(crawl_detail, link, prodi): link for link in links[:5]}
            for future in as_completed(future_to_link):
                data = future.result()
                if data:
                    prodi_results.append(data)
                if len(prodi_results) >= 5:
                    break

        # Tampilkan progress bar
        print_progress(i, prodi, 1, 1)
        sys.stdout.write("\n")
        # Gabungkan hasil prodi ke hasil akhir
        results.extend(prodi_results)

    # Simpan hasil ke DataFrame dan file CSV
    df = pd.DataFrame(results)
    df.to_csv("pta_all.csv", index=False, encoding="utf-8-sig")

    # Hitung waktu eksekusi
    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    # Tampilkan ringkasan hasil
    print("\n✅ Seluruh data berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [25]:
pta_all()

[1] Ilmu Hukum - Page 1/1 [████████████████████] 100.00%



[2] Teknologi Industri Pertanian - Page 1/1 [████████████████████] 100.00%



[3] Agribisnis - Page 1/1 [████████████████████] 100.00%



[4] Agroteknologi - Page 1/1 [████████████████████] 100.00%



[5] Ilmu Kelautan - Page 1/1 [████████████████████] 100.00%



[6] Ekonomi Pembangunan - Page 1/1 [████████████████████] 100.00%



[7] Manajemen - Page 1/1 [████████████████████] 100.00%



[8] Akuntansi - Page 1/1 [████████████████████] 100.00%



[9] Teknik Industri - Page 1/1 [████████████████████] 100.00%



[10] Teknik Informatika - Page 1/1 [████████████████████] 100.00%



[11] Manajemen Informatika - Page 1/1 [████████████████████] 100.00%



[12] Sosiologi - Page 1/1 [████████████████████] 100.00%



[13] Ilmu Komunikasi - Page 1/1 [████████████████████] 100.00%



[14] Psikologi - Page 1/1 [████████████████████] 100.00%



[15] Sastra Inggris - Page 1/1 [████████████████████] 100.00%



[16] Ekonomi Syariah - Page 1/1 

,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,080111100012,Dyah Ayu Citra Seza,Implementasi Fungsi Legislasi Dewan Perwakilan...,ABSTRAK\r\n\r\n Implementasi Fungsi Legi...,ABSTRACT\r\n Implementation of Legislati...,"Yudi Widagdo Harimurti, SH., MH","Safi', SH., MH",Ilmu Hukum
1,070111100060,Moh. Samsul Hidayat,Analisis Terhadap Kekosongan Hukum dalam Penga...,Kasus narkoba tidak henti-hentinya terdengar d...,"Drug cases endlessly heard on television, radi...","Tolib Effendi, SH., MH.","Agus Ramdlany, SH., MH.",Ilmu Hukum
2,080111100002,Maulina Nurlaily,Pertanggungjawaban Pidana Direksi BUMN (Perser...,Badan Usaha Milik Negara (BUMN) adalah Badan u...,State Owned Enterprises (SOEs) are business en...,"Tolib Effendi, SH., MH.","Dr. Eni Suastuti, SH., Mhum.",Ilmu Hukum
3,090111100077,TOMMY ADITYA PARLINDUNGAN MARBUN,PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK E...,Produk elektronik adalah suatu benda bergerak ...,Electronic products is an object moves through...,"DR. DJULAEKA, S.H., M.HUM","DR.USWATUN HASANAH, S.H., M. HUM",Ilmu Hukum
4,070111200007,RICA YENA IMADHORA,TELAAH KRITIS TENTANG ALASAN HUKUM YANG DIGUN...,,,"Dr. DENI SBY, S. H., M. S.","SAIFUL ABDULLAH, S. H., M. H.",Ilmu Hukum
...,...,...,...,...,...,...,...,...
163,160281100006,Sri Rohyatiningsih,PENGARUH TUNJANGAN KINERJA TERHADAP PRODUKTIVI...,Seiring dengan adanya tunjangan kinerja bagi p...,Along with the performance allowance for emplo...,"Dr. Mohtar Rasyid, S.E., M.Sc","Dr. Anita Kristina, S.E., M.Si",Magister Ilmu Ekonomi
164,160281100010,Moh. Syarif,ANALISIS PENENTUAN BESARAN DAN PENGELOLAAN DAN...,Dana Desa adalah dana yang bersumber dari Angg...,The Village Fund is a funding that sourced fro...,Eni Sri Rahayu Ningsih,Mohtar Rasyid,Magister Ilmu Ekonomi
165,170361100010,ahmad syaiful umam,KARAKTERISASI DAN KOLEKSI PLASMA NUTFAH UNTUK ...,Madura merupakan salah satu wilayah pemasok ko...,Madura is one of the regions supplying horticu...,"Dr. Ir. Gita Pawana, M.Si","Dr. Ir. Hj. SIti Fatimah, M.Si",Magister Pengelolaan Sumber Daya Alam
166,170361100001,Siti Holifah,PENGOLAHAN LIMBAH AIR REBUSAN IKAN TERI MENJAD...,Ikan Teri perlu penanganan serius pasca panen ...,Anchovy needs serious handling after harvest b...,"Dr.Apri Arisandi,S.Pi.,M.Si.","Dr.Ir.H.Asfan,MP.",Magister Pengelolaan Sumber Daya Alam
